In [8]:
import plot_probe_experiment_results as g
import importlib

importlib.reload(g)


results_datapath = 'probe_experiment_tracking.json'
model_prefixes=["big_random16M", "lichess_karvhyp"]
max_train_games_list=["20000"]
num_epochs_list=["3"],
probe_datasets=["lichess"]
training_configs=["classic"]
min_epochs = 3
max_epochs = 100

g.plot_accuracy_by_iterations(
    results_datapath,
    model_prefixes,
    training_configs,
    probe_datasets,
    max_train_games_list,
    min_epochs,
    max_epochs 
)


In [1]:
import plot_probe_experiment_results as g
import importlib

importlib.reload(g)


results_datapath = 'experiment_tracking.json'
model_prefixes=["big_random16M", "lichess_karvhyp"]
max_train_games_list=["20000"]
num_epochs_list=["3"],
probe_datasets=["lichess"]
training_configs=["classic"]
selection_arg="probe_dataset"

g.plot_accuracy_by_iterations(
    results_datapath,
    model_prefixes,
    max_train_games_list,
    num_epochs_list,
    probe_datasets,
    training_configs,
    selection_arg
)


FileNotFoundError: [Errno 2] No such file or directory: 'experiment_tracking.json'